In [117]:
!pip install keras
!pip install tensorflow

# IMAGE PREPROCESSING

In [118]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten

In [119]:
#Loading dataset into the cloud
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='67UVPINvQa8xHKQGFZ7G0S6A3_Yta4PBlnd8F1lOZr30',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'ecgarrythmia-donotdelete-pr-xnjamjasezeqom'
object_key = 'Classification of Arrhythmia by Using Deep Learning with 2-D ECG Spectral Image Representation.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [120]:
#unzipping your data file
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [121]:
pwd

'/home/wsuser/work'

In [122]:
import os
filenames=os.listdir('/home/wsuser/work/data/train')

In [123]:
 from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [124]:
# setting parameters for data augmentation for training data
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

In [125]:
# data augmentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

In [126]:
#performing data augmentation to train data
x_train=train_datagen.flow_from_directory(directory=r'/home/wsuser/work/data/train',target_size=(64,64),batch_size=32,class_mode='categorical')

Found 15341 images belonging to 6 classes.


In [127]:
x_test=test_datagen.flow_from_directory(directory=r'/home/wsuser/work/data/test',target_size=(64,64),batch_size=32,class_mode='categorical')

Found 6825 images belonging to 6 classes.


In [128]:
#Lets see the classes the different types of arrythmia is stored in
x_train.class_indices

{'Left Bundle Branch Block': 0,
 'Normal': 1,
 'Premature Atrial Contraction': 2,
 'Premature Ventricular Contractions': 3,
 'Right Bundle Branch Block': 4,
 'Ventricular Fibrillation': 5}

# MODEL BUILDING

MODEL BUILDING- IMPORTING LIBRARIES

In [129]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

from tensorflow.keras.layers import Dense,Flatten

from tensorflow.keras.layers import Conv2D,MaxPooling2D
import tensorflow.keras

In [130]:
model=Sequential()

Adding CNN Layers

In [131]:
model.add(Conv2D(32,(3,3),input_shape=(64,64,3),activation='relu')) #Activation Function
model.add(MaxPooling2D(pool_size=(2,2))) #Downsampling Purposes
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten()) #Flatten the dimensions of the image

ADDING DENSE LAYERS

In [132]:
model.add(Dense(32))# Deeply connected neural network layers
model.add(Dense(6,activation='softmax'))#Output layer with 6 neurons

In [133]:
#Summary of the model
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 6272)              0         
                                                                 
 dense_2 (Dense)             (None, 32)               

EVALUATING LOSS FUNCTION

In [134]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

TRAINING THE MODEL

In [135]:
model.fit_generator(generator=x_train,steps_per_epoch = len(x_train), epochs=10, validation_data=x_test,validation_steps = len(x_test))

/tmp/wsuser/ipykernel_165/53529210.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=x_train,steps_per_epoch = len(x_train), epochs=10, validation_data=x_test,validation_steps = len(x_test))


Epoch 1/10
480/480 [==============================] - 92s 191ms/step - loss: 0.8288 - accuracy: 0.7166 - val_loss: 0.4710 - val_accuracy: 0.8513
Epoch 2/10
480/480 [==============================] - 95s 197ms/step - loss: 0.2687 - accuracy: 0.9216 - val_loss: 0.3495 - val_accuracy: 0.8847
Epoch 3/10
480/480 [==============================] - 92s 192ms/step - loss: 0.2053 - accuracy: 0.9409 - val_loss: 0.3422 - val_accuracy: 0.8894
Epoch 4/10
480/480 [==============================] - 93s 194ms/step - loss: 0.1747 - accuracy: 0.9469 - val_loss: 0.2514 - val_accuracy: 0.9262
Epoch 5/10
480/480 [==============================] - 93s 194ms/step - loss: 0.1547 - accuracy: 0.9524 - val_loss: 0.2279 - val_accuracy: 0.9366
Epoch 6/10
480/480 [==============================] - 92s 191ms/step - loss: 0.1448 - accuracy: 0.9560 - val_loss: 0.2173 - val_accuracy: 0.9262
Epoch 7/10
480/480 [==============================] - 93s 193ms/step - loss: 0.1307 - accuracy: 0.9608 - val_loss: 0.2533 - val_ac

SAVING THE MODEL

In [136]:
model.save('ECG_Classification.h5')

In [137]:
!tar -zcvf ECG-Arrythmia-model_new.tgz ECG_Classification.h5

ECG_Classification.h5


In [138]:
ls -1

data/
ECG-arrhythmia-classification-model_new.tgz
ECG-Arrythmia-model_new.tgz
ECG-classification
ECG_Classification.h5
my_model.tar1.gz
my_model.tar.gz


In [139]:
#Installing amchine learning service
!pip install watson-machine-learning-client --upgrade

In [1]:
#Replace the credentials that you got from Watson Machine Learning Service
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"ge1RCI4GpBshW0Zu98SpfrdAdy6U4WbXKbiRUIxluG1O"
}
client=APIClient(wml_credentials)


In [141]:
client=APIClient(wml_credentials)

In [3]:
def guid_from_space_name(client,space_name):
    space=client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])


In [4]:
space_uid=guid_from_space_name(client, 'ECG-Arrythmia')
print("space UID="+ space_uid)

space UID=0680da27-17fb-450a-8b11-fc04d2de3272


In [5]:
client.set.default_space(space_uid)

'SUCCESS'

In [6]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1-py3.9             12b83a1

In [10]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [14]:
client.repository.download("73ecebcb-85ef-4e32-aabd-577361841231",'ECG.tar3.gz')

Successfully saved model content to file: 'ECG.tar3.gz'


'C:\\Users\\Adithyan\\Documents\\Python/ECG.tar3.gz'

TESTING THE MODEL

In [20]:
from keras.models import load_model
from keras.preprocessing import image


In [21]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='67UVPINvQa8xHKQGFZ7G0S6A3_Yta4PBlnd8F1lOZr30',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'ecgarrythmia-donotdelete-pr-xnjamjasezeqom'
object_key = 'pac.jpg'

streaming_body_3 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


EndpointConnectionError: Could not connect to the endpoint URL: "https://s3.private.us.cloud-object-storage.appdomain.cloud/ecgarrythmia-donotdelete-pr-xnjamjasezeqom/pac.jpg"

In [ ]:
#pwd

In [25]:
model=load_model("ECG-Arrythmia.h5")

OSError: No file or directory found at ECG-Arrythmia.h5

In [24]:
#Loading of image
img=image.load_img(r'/home/wsuser/work/data/test/Premature Ventricular Contractions/fig_5660.png',target_size=(64,64))
#Converting image to array
x=image.img_to_array(img)
#Changing the shape
x=np.expand_dims(x,axis=0)
#Predicting the classes
pred=model.predict(x)
y_pred=np.argmax(pred)
y_pred


FileNotFoundError: [Errno 2] No such file or directory: '/home/wsuser/work/data/test/Premature Ventricular Contractions/fig_5660.png'

In [178]:
index=['left Bundle Branch block','Normal','Premature Atrial Contraction','Premature Ventricular Contraction','Right Bundle Branch Block','Ventricular Fibrillation']
result = str(index[y_pred])
result

'Premature Ventricular Contraction'